In [1]:
import sys
import dbfread
from dbfread import DBF
import dbf
from glob import glob 
import pandas as pd
import csv
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import plotly as py
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
from datetime import datetime
import time
from collections import Counter
from tqdm import tqdm
import plotly.express as px

import matplotlib.font_manager as fm
# font_list = fm.findSystemFonts(fontpaths=None, fontext = 'ttf')

from matplotlib import font_manager, rc
# font_path = "C:/Windows/Fonts/NGULIM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
rc('font', family='AppleGothic')

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import layers, optimizers, models
from tensorflow.keras.layers import BatchNormalization, Bidirectional, Activation 
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.models import load_model
import tensorflow as tf

import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

import os

2024-08-12 10:29:12.900268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 10:29:12.926558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 10:29:12.956316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 10:29:12.965279: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 10:29:12.986905: I tensorflow/core/platform/cpu_feature_guar

## Combine features

In [2]:
result_label_path = os.path.expanduser('ALL_except_stress.csv')
result_nona = pd.read_csv(result_label_path)

In [3]:
result_nona

,eyebrow_distance_max,eye_eyebrow_distance_max,head_tilt_max,jaw_length_max,left_eye_height_max,right_eye_height_max,mouth_eye_distance_max,mouth_height_max,mouth_width_max,left_pupil_changes_max,...,ibi,bpm,sdnn,rmssd,pnn50,hr_mean,hr_std,stress,pnum,binarized
0,0.490723,0.236756,-0.228474,0.095122,0.532523,0.664515,0.567874,0.692026,0.596126,0.487650,...,-1.134372,1.125064,1.310402,-0.987050,0,0.862958,2.994207,0.872340,1,0
1,-0.713957,-0.428523,0.401581,0.105839,-0.331090,-0.528006,-0.408568,0.009546,-1.005822,-0.808883,...,1.750987,-1.599113,0.446963,1.070301,0,-1.396583,-0.540409,18.000000,1,1
2,-0.652070,-0.374610,0.401581,0.105839,-0.331090,-0.528006,-0.408568,0.009546,-1.005822,-0.808883,...,1.658159,-1.525154,0.585247,0.631118,0,-1.434242,-1.073155,18.000000,1,1
3,-0.652070,-0.374610,0.401581,0.105839,-0.331090,-0.476343,-0.222896,0.009546,-1.126854,-0.808883,...,1.548501,-1.436791,0.491494,0.686047,0,-1.446795,-1.092581,18.000000,1,1
4,-0.652070,-0.374610,0.358690,0.105839,-0.364570,-0.476343,-0.222896,0.009546,-1.126854,-0.702798,...,1.480392,-1.381358,0.092398,0.558550,0,-1.434242,-1.073155,18.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22116,0.348322,0.564742,0.183012,0.285776,-0.360708,-0.599946,0.234113,-0.414096,0.461679,0.125946,...,-0.763527,0.779800,-0.508828,-0.638931,0,0.643240,-0.064666,1.000000,45,0
22117,0.348322,0.523532,0.183012,0.285776,-0.360708,-0.617746,0.234113,-0.414096,0.461679,0.125946,...,-0.817688,0.843676,-0.636038,-0.678967,0,0.676260,-0.071251,1.000000,45,0
22118,0.298033,-0.109281,0.183012,0.243186,-0.360708,-0.617746,0.234113,-0.414096,0.458357,0.245446,...,-0.832583,0.861331,-0.666348,-0.704932,0,0.709280,-0.159521,1.000000,45,0
22119,0.304875,0.514718,0.289202,0.487044,0.109666,0.121898,0.510953,0.068908,0.431887,0.046539,...,-0.393957,0.357188,-0.250600,-0.318802,0,0.824850,0.013881,1.000000,45,0


# Label Distribution

In [4]:
# customer_f0_max 기준으로 나누기
customer_f0_max_index = result_nona.columns.get_loc('customer_f0_max')

# EDA_mean을 기준으로 나누기
eda_index = result_nona.columns.get_loc('EDA_mean')

# label을 기준으로 나누기
#label_index = result_nona.columns.get_loc('suppress')
#label_index = result_nona.columns.get_loc('call_label')
label_index = result_nona.columns.get_loc('stress')
# video
# 처음부터 customer_f0_max_index까지의 열 선택
first_part = result_nona.iloc[:, :customer_f0_max_index]
# suppress 열부터 끝까지의 열 선택
second_part = result_nona.iloc[:, label_index:]
# 두 부분을 열 방향으로 결합
X_video = pd.concat([first_part, second_part], axis=1)

# audio
# customer_f0_max_index부터 eda_index까지의 열 선택
first_part = result_nona.iloc[:, customer_f0_max_index:eda_index]
# suppress 열부터 끝까지의 열 선택
second_part = result_nona.iloc[:, label_index:]
# 두 부분을 열 방향으로 결합
X_audio = pd.concat([first_part, second_part], axis=1)

# physiological 
X_physi = result_nona.iloc[:,  eda_index:]

In [5]:
X_video

,eyebrow_distance_max,eye_eyebrow_distance_max,head_tilt_max,jaw_length_max,left_eye_height_max,right_eye_height_max,mouth_eye_distance_max,mouth_height_max,mouth_width_max,left_pupil_changes_max,...,right_pupil_changes_std,blinking_std,smile_std,hand_face_distance_std,shoulder_position_changes_std,shoulder_angle_changes_std,blinking_count,stress,pnum,binarized
0,0.490723,0.236756,-0.228474,0.095122,0.532523,0.664515,0.567874,0.692026,0.596126,0.487650,...,0.819396,1.672163,0.465240,-0.694466,-0.274094,-0.356413,2.269855,0.872340,1,0
1,-0.713957,-0.428523,0.401581,0.105839,-0.331090,-0.528006,-0.408568,0.009546,-1.005822,-0.808883,...,-0.428278,0.971497,-0.715298,-0.255088,-0.695030,-1.018556,0.898801,18.000000,1,1
2,-0.652070,-0.374610,0.401581,0.105839,-0.331090,-0.528006,-0.408568,0.009546,-1.005822,-0.808883,...,-0.466578,1.131356,-0.715298,0.017440,-0.725372,-1.045725,1.173012,18.000000,1,1
3,-0.652070,-0.374610,0.401581,0.105839,-0.331090,-0.476343,-0.222896,0.009546,-1.126854,-0.808883,...,-0.461392,0.797811,-0.247567,0.291461,-0.713758,-1.176867,0.624591,18.000000,1,1
4,-0.652070,-0.374610,0.358690,0.105839,-0.364570,-0.476343,-0.222896,0.009546,-1.126854,-0.702798,...,-0.404996,0.606844,-0.110907,0.236211,-0.707991,-1.277823,0.350380,18.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22116,0.348322,0.564742,0.183012,0.285776,-0.360708,-0.599946,0.234113,-0.414096,0.461679,0.125946,...,0.563336,0.933870,0.948182,-0.221614,0.448057,0.453306,1.061206,1.000000,45,0
22117,0.348322,0.523532,0.183012,0.285776,-0.360708,-0.617746,0.234113,-0.414096,0.461679,0.125946,...,0.593217,0.979128,1.289625,-0.221614,0.658228,0.551481,1.061206,1.000000,45,0
22118,0.298033,-0.109281,0.183012,0.243186,-0.360708,-0.617746,0.234113,-0.414096,0.458357,0.245446,...,0.895224,0.979128,1.341479,-0.221614,0.972311,0.976031,1.061206,1.000000,45,0
22119,0.304875,0.514718,0.289202,0.487044,0.109666,0.121898,0.510953,0.068908,0.431887,0.046539,...,-0.140454,-0.645096,0.899469,-0.221614,0.404250,0.151828,-0.838655,1.000000,45,0


In [6]:
X_audio

,customer_f0_max,customer_voiced_flag_max,customer_energy_max,customer_zcr_max,customer_cent_max,customer_spec_bw_max,customer_rolloff_max,customer_rolloff_min_max,customer_contrast_max,customer_mfcc_1_max,...,worker_mfcc_7_std,worker_mfcc_8_std,worker_mfcc_9_std,worker_mfcc_10_std,worker_mfcc_11_std,worker_mfcc_12_std,worker_mfcc_13_std,stress,pnum,binarized
0,4.886489,0.711098,-0.495223,-0.563020,-0.569009,-0.687802,-0.639507,1.225657,-0.144170,-0.268526,...,-0.247756,-0.808253,0.050627,0.624671,0.150429,-0.789598,-0.346970,0.872340,1,0
1,-0.485906,-1.406276,-0.535877,-0.648263,-0.523181,-0.495498,-0.090769,-1.247426,-0.281214,-0.499138,...,-0.232140,0.393532,-0.205968,-0.684879,-0.588394,-0.478135,-0.175389,18.000000,1,1
2,-0.485906,-1.406276,-0.535877,-0.392533,-0.523181,-0.495498,-0.090769,-1.247426,-0.281214,-0.499138,...,-0.081997,0.630663,0.073423,-0.347569,-0.364614,-0.119237,-0.109337,18.000000,1,1
3,-0.485906,-1.406276,-0.535877,-0.392533,-0.523181,-0.467349,-0.090769,-1.126788,-0.629371,-0.499138,...,-0.019547,0.711173,0.203126,-0.271239,-0.195012,-0.090525,-0.078483,18.000000,1,1
4,-0.485906,-1.406276,-0.535877,-0.364118,-0.642258,-0.467349,-0.639507,-1.126788,-0.629371,-0.499138,...,-0.482931,0.506030,0.260372,-0.302384,-0.491830,-0.375135,-0.281246,18.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22116,-0.044566,0.565793,-0.410856,1.133355,-0.577353,-0.485240,-0.165189,0.112267,-0.742955,0.258621,...,-0.100728,0.777674,-0.226023,-0.227059,0.066884,0.045263,0.643541,1.000000,45,0
22117,0.507327,0.565793,-0.332795,0.961676,-0.577353,-0.485240,-0.165189,-0.435691,-0.742955,0.225621,...,-0.366279,-0.298979,-0.621508,-0.646336,0.243834,0.012565,0.388676,1.000000,45,0
22118,0.507327,0.565793,-0.332795,0.961676,-0.276059,-0.485240,-0.165189,-0.435691,-0.742955,0.017787,...,-0.325476,-0.669973,-0.398855,-0.492038,0.413386,0.128803,0.624176,1.000000,45,0
22119,0.052250,0.565793,-0.034271,1.324108,-0.474910,-0.342625,-0.165189,0.190547,0.288531,0.462760,...,0.115511,0.010740,-0.320064,0.027302,-0.094019,-0.162489,-0.694270,1.000000,45,0


In [7]:
X_physi

,EDA_mean,EDA_std,EDA_min,EDA_max,phasic_mean,phasic_std,phasic_min,phasic_max,tonic_mean,tonic_std,...,ibi,bpm,sdnn,rmssd,pnn50,hr_mean,hr_std,stress,pnum,binarized
0,-3.162248,-1.114870,-3.180907,-2.973726,-0.945928,-1.035553,-0.649199,-1.092026,-3.385110,-0.269027,...,-1.134372,1.125064,1.310402,-0.987050,0,0.862958,2.994207,0.872340,1,0
1,0.156475,-0.245898,0.343829,0.087816,-0.281320,0.428472,-0.534304,0.077503,0.256216,0.314344,...,1.750987,-1.599113,0.446963,1.070301,0,-1.396583,-0.540409,18.000000,1,1
2,0.128172,-0.132128,0.089950,0.087816,-0.205235,0.404473,-0.534304,0.077503,0.209007,0.470575,...,1.658159,-1.525154,0.585247,0.631118,0,-1.434242,-1.073155,18.000000,1,1
3,0.084197,0.082868,0.001388,0.087816,-0.158089,0.361720,-0.534304,0.077503,0.157820,0.604005,...,1.548501,-1.436791,0.491494,0.686047,0,-1.446795,-1.092581,18.000000,1,1
4,0.044006,0.250368,0.001388,0.087816,-0.129148,0.324051,-0.534304,0.077503,0.103399,0.711091,...,1.480392,-1.381358,0.092398,0.558550,0,-1.434242,-1.073155,18.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22116,1.819809,2.007330,1.372457,1.860088,4.206565,1.257967,9.062338,2.491703,1.095611,5.076620,...,-0.763527,0.779800,-0.508828,-0.638931,0,0.643240,-0.064666,1.000000,45,0
22117,1.897049,1.677261,1.525346,1.860088,3.968365,1.580391,6.349432,2.491703,1.226316,5.022234,...,-0.817688,0.843676,-0.636038,-0.678967,0,0.676260,-0.071251,1.000000,45,0
22118,1.944086,1.317152,1.525346,1.860088,3.667836,1.993601,4.476786,2.491703,1.353878,4.803244,...,-0.832583,0.861331,-0.666348,-0.704932,0,0.709280,-0.159521,1.000000,45,0
22119,0.513396,2.183817,0.347663,1.082368,1.178181,1.966067,0.275512,1.849480,0.312349,1.242253,...,-0.393957,0.357188,-0.250600,-0.318802,0,0.824850,0.013881,1.000000,45,0


# audio Kernel

In [8]:
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score

#tensor로 변환
import torch
from MKLpy.utils.validation import check_X

#RGF generator
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel, linear_kernel, euclidean_distances

#base learner
from sklearn.svm import SVC
from MKLpy.algorithms import AverageMKL

#evaluation
from sklearn.metrics import accuracy_score, roc_auc_score

#병렬 처리
from joblib import Parallel, delayed

## 병렬처리

In [9]:
results_filename = 'ex_rbf_g001_c01_all_stre.csv'

# Initialize result storage DataFrame
all_results = []

#성능 측정을 위한 데이터프레임 초기화
three_performance_mean = pd.DataFrame(columns=['accuracy', 'f1_macro', 'f1_micro', 'auc_ovr'])
three_performance_std = pd.DataFrame(columns=['accuracy', 'f1_macro', 'f1_micro', 'auc_ovr'])

binary_performance_mean = pd.DataFrame(columns=['accuracy', 'f1_score', 'auc'])
binary_performance_std = pd.DataFrame(columns=['accuracy', 'f1_score', 'auc'])


call_three_label = False
call_label = False

In [10]:
#for video
def train_and_evaluate(train_index, test_index, X_video_features, X_audio_features, X_physi_features, y_label, PIDS):
    # for train_index, test_index in splitter.split(X_audio_features, y_label, PIDS):
        X_video_train, X_video_test = X_video_features.iloc[train_index], X_video_features.iloc[test_index]
        X_audio_train, X_audio_test = X_audio_features.iloc[train_index], X_audio_features.iloc[test_index]
        X_physi_train, X_physi_test = X_physi_features.iloc[train_index], X_physi_features.iloc[test_index]
        y_train, y_test = y_label.iloc[train_index].values, y_label.iloc[test_index].values
        test_name = np.unique(PIDS[test_index])[0]
                
        print("start")
        print(test_name)

        #feature 중요도 처리
        columns = ['hand_face_distance_max', 'left_eye_height_max', 'hand_face_distance_mean', 'smile_min', 'hand_face_distance_min']
    
        # feature data -> tensor로 변환
        X_numpy = X_video_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_video_train = check_X(X_tensor)

        X_numpy = X_audio_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_audio_train = check_X(X_tensor)
        
        X_numpy = X_physi_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_physi_train = check_X(X_tensor)
        
        # train kernel 생성 (hyperparameter: gamma)
        K_video_train = rbf_kernel(X_video_train, gamma=0.01)
        K_audio_train = rbf_kernel(X_audio_train, gamma=0.01)
        K_physi_train = rbf_kernel(X_physi_train, gamma=0.01)
    
        # train kernel 결합
        K_train = [K_video_train, K_audio_train, K_physi_train]
    
        # test kernel 생성 (hyperparameter: gamma)
        K_video_test = rbf_kernel(X_video_test, X_video_train, gamma=0.01)
        K_audio_test = rbf_kernel(X_audio_test, X_audio_train, gamma=0.01)
        K_physi_test = rbf_kernel(X_physi_test, X_physi_train, gamma=0.01)

        # test kernel 결합
        K_test = [K_audio_test, K_video_test, K_physi_test]

        #MKL 설정
        base_learner = SVC(C=0.1)
        mkl = AverageMKL(learner=base_learner).fit(K_train, y_train)
        
        # predict
        y_pred  = mkl.predict(K_test)            #predict the output class
        y_score = mkl.decision_function(K_test)  #returns the projection on the distance vector
        
        
        if call_three_label:
            result = {
                'accuracy': accuracy_score(y_test, y_pred),
                'confusion_matrix': confusion_matrix(y_test, y_pred),
                'f1_macro': f1_score(y_test, y_pred, average='macro'),
                'f1_micro': f1_score(y_test, y_pred, average='micro'),
                'f1_weighted': f1_score(y_test, y_pred, average='weighted'),
                'auc_ovo': roc_auc_score(y_test, y_score, multi_class='ovo'),
                'auc_ovr': roc_auc_score(y_test, y_score, multi_class='ovr'),
                'test_name': 'p' + str(test_name)
            }
        else:
            result = {
                'accuracy': accuracy_score(y_test, y_pred),
                'confusion_matrix': confusion_matrix(y_test, y_pred),
                'f1_score': f1_score(y_test, y_pred),
                'auc': roc_auc_score(y_test, y_score),
                'test_name': 'p' + str(test_name)
            }


        RESULTS_AVAL_SVC.append(result)
        # 결과를 파일에 기록
        result_df = pd.DataFrame([result])
        header = not os.path.exists(results_filename) or os.stat(results_filename).st_size == 0
        result_df.to_csv(results_filename, mode='a', header=header, index=False)

        print(RESULTS_AVAL_SVC)
        print("finish")

        test.append(y_pred)
        pred.append(y_test)
        #importance.append(mkl.feature_importances_)        


In [ ]:
n_jobs = 10

for i in range(3):
    X_video_features= pd.DataFrame()
    X_audio_features= pd.DataFrame()
    X_physi_features= pd.DataFrame()
    
    if call_label:
        X_video_features= X_video.drop(['pnum', 'call_label'], axis=1)
        X_audio_features= X_audio.drop(['pnum', 'call_label'], axis=1)
        X_physi_features= X_physi.drop(['pnum', 'call_label'], axis=1)
        y_label = result_nona['call_label']
    elif call_three_label: 
        X_video_features= X_video.drop(['pnum', 'call_three_label'], axis=1)
        X_audio_features= X_audio.drop(['pnum', 'call_three_label'], axis=1)
        X_physi_features= X_physi.drop(['pnum', 'call_three_label'], axis=1)
        y_label = result_nona['call_three_label']
    else:
        X_video_features= X_video.drop(['stress', 'pnum', 'binarized'], axis=1)
        X_audio_features= X_audio.drop(['stress', 'pnum', 'binarized'], axis=1)
        X_physi_features= X_physi.drop(['stress', 'pnum', 'binarized'], axis=1)

        # X_video_features= X_video.drop(['suppress', 'pnum', 'binarized'], axis=1)
        # X_audio_features= X_audio.drop(['suppress', 'pnum', 'binarized'], axis=1)
        # X_physi_features= X_physi.drop(['suppress', 'pnum', 'binarized'], axis=1)
        y_label = result_nona['binarized']
        
    PIDS = result_nona['pnum']

    test = []
    pred = []
    #importance = [] 

    RESULTS_AVAL_SVC, MODELS_AVAL_SVC, TRAIN_SETS_AVAL_SVC = [], {}, {}
    splitter = LeaveOneGroupOut()
    
    # 병렬처리
    results = Parallel(n_jobs=n_jobs, backend='threading')(
       delayed(train_and_evaluate)(train_index, test_index, X_video_features, X_audio_features, X_physi_features, y_label, PIDS)
       for train_index, test_index in splitter.split(X_audio_features, y_label, PIDS)
    #    for I_train, I_test in splitter.split(X_features, y_label, PIDS)
   )


start
1
start
8
start
3
start
4
start
12
start
10
start
14
start
2
start
16
start
13


In [ ]:
results_filename = 'ex_rbf_g001_c01_physi_stre.csv'

# Initialize result storage DataFrame
all_results = []

#성능 측정을 위한 데이터프레임 초기화
three_performance_mean = pd.DataFrame(columns=['accuracy', 'f1_macro', 'f1_micro', 'auc_ovr'])
three_performance_std = pd.DataFrame(columns=['accuracy', 'f1_macro', 'f1_micro', 'auc_ovr'])

binary_performance_mean = pd.DataFrame(columns=['accuracy', 'f1_score', 'auc'])
binary_performance_std = pd.DataFrame(columns=['accuracy', 'f1_score', 'auc'])


call_three_label = False
call_label = False

In [ ]:
#for video
def train_and_evaluate(train_index, test_index, X_video_features, X_audio_features, X_physi_features, y_label, PIDS):
    # for train_index, test_index in splitter.split(X_audio_features, y_label, PIDS):
        X_video_train, X_video_test = X_video_features.iloc[train_index], X_video_features.iloc[test_index]
        X_audio_train, X_audio_test = X_audio_features.iloc[train_index], X_audio_features.iloc[test_index]
        X_physi_train, X_physi_test = X_physi_features.iloc[train_index], X_physi_features.iloc[test_index]
        y_train, y_test = y_label.iloc[train_index].values, y_label.iloc[test_index].values
        test_name = np.unique(PIDS[test_index])[0]
                
        print("start")
        print(test_name)

        #feature 중요도 처리
        columns = ['hand_face_distance_max', 'left_eye_height_max', 'hand_face_distance_mean', 'smile_min', 'hand_face_distance_min']
    
        # feature data -> tensor로 변환
        X_numpy = X_video_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_video_train = check_X(X_tensor)

        X_numpy = X_audio_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_audio_train = check_X(X_tensor)
        
        X_numpy = X_physi_train.values
        X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
        X_physi_train = check_X(X_tensor)
        
        # train kernel 생성 (hyperparameter: gamma)
        K_video_train = rbf_kernel(X_video_train, gamma=0.01)
    
        # train kernel 결합
        K_train = [K_video_train]
    
        # test kernel 생성 (hyperparameter: gamma)
        K_video_test = rbf_kernel(X_video_test, X_video_train, gamma=0.01)

        # test kernel 결합
        K_test = [K_video_test]

        #MKL 설정
        base_learner = SVC(C=0.1)
        mkl = AverageMKL(learner=base_learner).fit(K_train, y_train)
        
        # predict
        y_pred  = mkl.predict(K_test)            #predict the output class
        y_score = mkl.decision_function(K_test)  #returns the projection on the distance vector
        
        
        if call_three_label:
            result = {
                'accuracy': accuracy_score(y_test, y_pred),
                'confusion_matrix': confusion_matrix(y_test, y_pred),
                'f1_macro': f1_score(y_test, y_pred, average='macro'),
                'f1_micro': f1_score(y_test, y_pred, average='micro'),
                'f1_weighted': f1_score(y_test, y_pred, average='weighted'),
                'auc_ovo': roc_auc_score(y_test, y_score, multi_class='ovo'),
                'auc_ovr': roc_auc_score(y_test, y_score, multi_class='ovr'),
                'test_name': 'p' + str(test_name)
            }
        else:
            result = {
                'accuracy': accuracy_score(y_test, y_pred),
                'confusion_matrix': confusion_matrix(y_test, y_pred),
                'f1_score': f1_score(y_test, y_pred),
                'auc': roc_auc_score(y_test, y_score),
                'test_name': 'p' + str(test_name)
            }


        RESULTS_AVAL_SVC.append(result)
        # 결과를 파일에 기록
        result_df = pd.DataFrame([result])
        header = not os.path.exists(results_filename) or os.stat(results_filename).st_size == 0
        result_df.to_csv(results_filename, mode='a', header=header, index=False)

        print(RESULTS_AVAL_SVC)
        print("finish")

        test.append(y_pred)
        pred.append(y_test)
        #importance.append(mkl.feature_importances_)        


In [ ]:
n_jobs = 10

for i in range(3):
    X_video_features= pd.DataFrame()
    X_audio_features= pd.DataFrame()
    X_physi_features= pd.DataFrame()
    
    if call_label:
        X_video_features= X_video.drop(['pnum', 'call_label'], axis=1)
        X_audio_features= X_audio.drop(['pnum', 'call_label'], axis=1)
        X_physi_features= X_physi.drop(['pnum', 'call_label'], axis=1)
        y_label = result_nona['call_label']
    elif call_three_label: 
        X_video_features= X_video.drop(['pnum', 'call_three_label'], axis=1)
        X_audio_features= X_audio.drop(['pnum', 'call_three_label'], axis=1)
        X_physi_features= X_physi.drop(['pnum', 'call_three_label'], axis=1)
        y_label = result_nona['call_three_label']
    else:
        # X_video_features= X_video.drop(['stress', 'pnum', 'binarized'], axis=1)
        # X_audio_features= X_audio.drop(['stress', 'pnum', 'binarized'], axis=1)
        # X_physi_features= X_physi.drop(['stress', 'pnum', 'binarized'], axis=1)

        X_video_features= X_video.drop(['suppress', 'pnum', 'binarized'], axis=1)
        X_audio_features= X_audio.drop(['suppress', 'pnum', 'binarized'], axis=1)
        X_physi_features= X_physi.drop(['suppress', 'pnum', 'binarized'], axis=1)
        y_label = result_nona['binarized']
        
    PIDS = result_nona['pnum']

    test = []
    pred = []
    #importance = [] 

    RESULTS_AVAL_SVC, MODELS_AVAL_SVC, TRAIN_SETS_AVAL_SVC = [], {}, {}
    splitter = LeaveOneGroupOut()
    
    # 병렬처리
    results = Parallel(n_jobs=n_jobs, backend='threading')(
       delayed(train_and_evaluate)(train_index, test_index, X_video_features, X_audio_features, X_physi_features, y_label, PIDS)
       for train_index, test_index in splitter.split(X_audio_features, y_label, PIDS)
    #    for I_train, I_test in splitter.split(X_features, y_label, PIDS)
   )
